In [ ]:
import os
import random

import cv2
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import torchvision.transforms.functional as F_v
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import ConcatDataset, DataLoader, Dataset, WeightedRandomSampler
from torchvision import models
from tqdm.auto import tqdm

RANDOM_SEED = 42

# Python, NumPy, and Torch seed, ensure deterministic behaviour
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)

In [ ]:
TRAIN_METADATA_CSV = "/kaggle/input/kaggleisic-challenge/new-train-metadata.csv"
TEST_METADATA_CSV = "/kaggle/input/kaggleisic-challenge/students-test-metadata.csv"
TRAIN_METADATA_PROCESSED_CSV = (
    "/kaggle/input/kaggleisic-challenge/train-metadata-processed.csv"
)
TEST_METADATA_PROCESSED_CSV = (
    "/kaggle/input/kaggleisic-challenge/test-metadata-processed.csv"
)
TRAIN_HDF5 = "/kaggle/input/kaggleisic-challenge/train-image.hdf5"
TEST_HDF5 = "/kaggle/input/kaggleisic-challenge/test-image.hdf5"

TRAIN_METADATA_AUGMENTED_CSV = (
    "/kaggle/input/kaggleisic-challenge/train-metadata-augmented.csv"
)
TRAIN_AUGMENTED_HDF5 = "/kaggle/input/kaggleisic-challenge/train-image-augmented.hdf5"

OUTPUT_FINAL_MODEL = "/kaggle/working/final_model.pth"
OUTPUT_FINAL_SUBMISSION = "/kaggle/working/final_submission.csv"

DROP_COLUMNS = [
    "image_type",
    "patient_id",
    "copyright_license",
    "attribution",
    "anatom_site_general",
    "tbp_lv_location_simple",
]

In [ ]:
class ISIC_HDF5_Dataset(Dataset):
    """
    A PyTorch Dataset that loads images from an HDF5 file given a DataFrame of IDs.
    Applies image transforms.
    """

    def __init__(
        self, df: pd.DataFrame, hdf5_path: str, transform=None, is_labelled: bool = True
    ):
        """
        Args:
            df (pd.DataFrame): DataFrame containing 'isic_id' and optionally 'target'.
            hdf5_path (str): Path to the HDF5 file containing images.
            transform (callable): Optional transforms to be applied on a sample.
            is_labelled (bool): Whether the dataset includes labels (for train/val).
        """
        self.df = df.reset_index(drop=True)
        self.hdf5_path = hdf5_path
        self.transform = transform
        self.is_labelled = is_labelled

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        isic_id = row["isic_id"]

        # Load image from HDF5
        image_rgb = self._load_image_from_hdf5(isic_id)

        # Apply transforms (PIL-style transforms require converting np array to PIL, or we can do tensor transforms)
        if self.transform is not None:
            # Convert NumPy array (H x W x C) to a PIL Imag
            image_pil = F_v.to_pil_image(image_rgb)
            image = self.transform(image_pil)
        else:
            # By default, convert it to a PIL Image
            view_transform = T.Compose([T.Resize((224, 224)), T.ToTensor()])
            image_pil = F_v.to_pil_image(image_rgb)
            image = view_transform(image_pil)

        if self.is_labelled:
            label = row["target"]
            label = torch.tensor(label).float()
            return image, label, isic_id
        else:
            return image, isic_id

    def _load_image_from_hdf5(self, isic_id: str):
        """
        Loads and decodes an image from HDF5 by isic_id.
        Returns a NumPy array in RGB format (H x W x 3).
        """
        with h5py.File(self.hdf5_path, "r") as hf:
            encoded_bytes = hf[isic_id][()]  # uint8 array

        # Decode the image bytes with OpenCV (returns BGR)
        image_bgr = cv2.imdecode(encoded_bytes, cv2.IMREAD_COLOR)
        # Convert to RGB
        image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        return image_rgb

In [ ]:
def load_metadata_dataset_only_healthy(
    train_frac=0.8, seed=42, is_subsampled=False, is_augmented=False
) -> tuple:
    if is_augmented:
        train_file = TRAIN_METADATA_AUGMENTED_CSV
    else:
        train_file = TRAIN_METADATA_PROCESSED_CSV

    # Load the metadata CSV files
    train_df = pd.read_csv(train_file)
    test_df = pd.read_csv(TEST_METADATA_PROCESSED_CSV)

    # Filter out healthy cases
    train_df = train_df[train_df["target"] == 0].reset_index(drop=True)

    # Perform stratified train/validation split to maintain class distribution
    train_dataset, valid_dataset = train_test_split(
        train_df, train_size=train_frac, stratify=train_df["target"], random_state=seed
    )

    # Reset index for train and validation datasets
    train_dataset = train_dataset.reset_index(drop=True)
    valid_dataset = valid_dataset.reset_index(drop=True)
    test_dataset = test_df.reset_index(drop=True)

    # Optionally create a balanced subset
    if is_subsampled:
        train_dataset = create_balanced_subset(train_dataset)
        valid_dataset = create_balanced_subset(valid_dataset)

    print(f"train_dataset shape: {train_dataset.shape}")
    print(f"valid_dataset shape: {valid_dataset.shape}")
    print(f"test_dataset shape:  {test_dataset.shape}")

    return train_dataset, valid_dataset, test_dataset


def load_hdf5_dataset(
    transform: T.Compose,
    train_frac=0.8,
    seed=42,
    is_subsampled=False,
    is_augmented=False,
) -> tuple[ISIC_HDF5_Dataset]:
    """
    Load the ISIC dataset from HDF5 files and split it into train, validation, and test sets.
    Args:
        transform (T.Compose): Transformations to apply to the images.
        train_frac (float): Fraction of the dataset to use for training.
        seed (int): Random seed for reproducibility.
    Returns:
        tuple: A tuple containing the train, validation, and test datasets.
    """
    # Load the metadata CSV files
    train_df_sub, valid_df_sub, test_df = load_metadata_dataset_only_healthy(
        train_frac=train_frac,
        seed=seed,
        is_subsampled=is_subsampled,
        is_augmented=is_augmented,
    )

    if is_augmented:
        train_file = TRAIN_AUGMENTED_HDF5
    else:
        train_file = TRAIN_HDF5

    # Create Datasets
    train_dataset = ISIC_HDF5_Dataset(
        df=train_df_sub, hdf5_path=train_file, transform=transform, is_labelled=True
    )

    valid_dataset = ISIC_HDF5_Dataset(
        df=valid_df_sub, hdf5_path=train_file, transform=transform, is_labelled=True
    )

    test_dataset = ISIC_HDF5_Dataset(
        df=test_df, hdf5_path=TEST_HDF5, transform=transform, is_labelled=False
    )

    return train_dataset, valid_dataset, test_dataset


def create_balanced_subset(
    df: pd.DataFrame, target_col="target", seed=42
) -> pd.DataFrame:
    # Just keep all the cancer cases and subsample the healthy cases (2:1 ratio)
    positives = df[df[target_col] == 1]

    n_negatives = len(positives) * 2  # 2:1 ratio
    negatives = df[df[target_col] == 0].sample(
        n=min(n_negatives, len(df[df[target_col] == 0])), random_state=seed
    )
    balanced_df = (
        pd.concat([positives, negatives])
        .sample(frac=1, random_state=seed)
        .reset_index(drop=True)
    )
    return balanced_df

In [ ]:
train_meta_df, valid_meta_df, test_meta_df = load_metadata_dataset_only_healthy(
    is_subsampled=True, is_augmented=False
)

In [ ]:
view_transform = T.Compose(
    [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
train_hdf5_dataset, valid_hdf5_dataset, test_hdf5_dataset = load_hdf5_dataset(
    transform=view_transform, is_subsampled=True, is_augmented=False
)

In [ ]:
BATCH_SIZE = 8  # batch size
NUM_SAMPLES = 1_000  # samples per epoch
NUM_WORKERS = 0

class_counts = train_meta_df["target"].value_counts().sort_index()
class_weights = 1.0 / class_counts

# Normalize weights to sum to 1
class_weights = class_weights / class_weights.sum()

sample_weights = train_meta_df["target"].map(class_weights).values

sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=NUM_SAMPLES,
    replacement=True,
)

In [ ]:
EPOCHS = 50
LEARNING_RATE = 1e-3
SCHEDULER_STEP_SIZE = 4
SCHEDULER_GAMMA = 0.5
MIN_DELTA = 0.001


def train_valid(model, train_loader, valid_loader, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = StepLR(optimizer, step_size=SCHEDULER_STEP_SIZE, gamma=SCHEDULER_GAMMA)

    # Tracking lists
    train_loses = []
    valid_loses = []

    best_valid_loss = float("inf")
    epochs_no_improve = 0

    for epoch in range(1, EPOCHS + 1):
        train_loss = train_vae(model, device, train_loader, optimizer, epoch)
        valid_loss = validate_vae(model, device, valid_loader, epoch)

        train_loses.append(train_loss)
        valid_loses.append(valid_loss)

        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        print(f"Learning Rate: {current_lr}")

        # Early Stopping check with threshold
        if valid_loss > best_valid_loss + MIN_DELTA:
            best_valid_loss = valid_loss
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            print(
                f"No improvement in {epochs_no_improve} epochs (threshold of {MIN_DELTA})."
            )

        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {epoch} epochs!")
            break

    # Plot training and validation ROC AUC scores
    plot_train_valid_curves(train_loses, valid_loses)

    print("Training complete ✅")

    return epoch


def train_eval(
    model,
    full_loader,
    test_loader,
    early_stopping_epochs=EPOCHS,
    output_model_file="model_final.pth",
    output_submission_file="submission.csv",
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = StepLR(optimizer, step_size=SCHEDULER_STEP_SIZE, gamma=SCHEDULER_GAMMA)

    # Tracking lists
    train_losses = []

    for epoch in range(1, early_stopping_epochs + 1):
        train_loss = train_vae(model, device, full_loader, optimizer, epoch)
        train_losses.append(train_loss)

        scheduler.step()
        current_lr = scheduler.get_last_lr()[0]
        print(f"Learning Rate: {current_lr}")

    # Save final model
    output_model_path = output_model_file
    torch.save(model.state_dict(), output_model_path)
    print(f"Model saved to {output_model_path}")

    # Plot training ROC AUC scores
    plot_train_curves(train_losses)

    print("Training complete ✅")

    # Evaluate on test set
    submission_df = evaluate_vae(model, device, test_loader)

    # Save submission file
    submission_file_path = output_submission_file
    submission_df.to_csv(submission_file_path, index=False)

    print(
        f"Saved submission with {len(submission_df)} rows to {submission_file_path} ✅"
    )


def train_vae(model, device, train_loader, optimizer, epoch):
    model.train()
    running_loss = 0.0

    for singles, _, _ in tqdm(train_loader, desc=f"Train Epoch {epoch}", leave=False):
        singles = singles.to(device)

        optimizer.zero_grad()

        recon_images, mu, logvar = model(singles)

        loss = model.loss_function(recon_images, singles, mu, logvar)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    print(f"Epoch {epoch}/{EPOCHS} | Train Loss: {avg_train_loss:.4f}")
    return avg_train_loss


def validate_vae(model, device, valid_loader, epoch):
    model.eval()
    val_loss = 0.0

    with torch.no_grad():
        for singles, _, _ in tqdm(
            valid_loader, desc=f"Validation Epoch {epoch}", leave=False
        ):
            singles = singles.to(device)

            recon_images, mu, logvar = model(singles)

            # Compute the loss
            loss = model.loss_function(recon_images, singles, mu, logvar)

            val_loss += loss.item()

    avg_val_loss = val_loss / len(valid_loader)

    print(f"Epoch {epoch}/{EPOCHS} | Validation Loss: {avg_val_loss:.4f}")
    return avg_val_loss


def evaluate_vae(model, device, test_loader):
    model.eval()
    predictions = []

    with torch.no_grad():
        for singles, isic_ids in tqdm(test_loader, desc="Inference on Test"):
            singles = singles.to(device)

            # Forward pass
            recon_images, mu, logvar = model(singles)

            # Calculate reconstruction error (mean squared error for anomaly detection)
            reconstruction_error = F.mse_loss(recon_images, singles, reduction="none")
            reconstruction_error = reconstruction_error.view(singles.size(0), -1).mean(
                dim=1
            )

            # Threshold for anomaly detection
            probability = torch.sigmoid(reconstruction_error).cpu().numpy()
            predictions.extend(zip(isic_ids, probability))

    submission_df = pd.DataFrame(predictions, columns=["isic_id", "target"])
    submission_df = submission_df.sort_values(by="isic_id").reset_index(drop=True)

    return submission_df


def plot_train_valid_curves(train_losses, valid_losses):
    # Prepare DataFrame for seaborn
    epochs = list(range(1, len(train_losses) + 1))
    data = pd.DataFrame(
        {
            "Epoch": epochs * 2,
            "Loss": train_losses + valid_losses,
            "Phase": ["Train"] * len(train_losses) + ["Validation"] * len(valid_losses),
        }
    )

    # Plot with seaborn
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=data, x="Epoch", y="Loss", hue="Phase", marker="o")
    plt.title("Training vs Validation Loss")
    plt.tight_layout()
    plt.show()


def plot_train_curves(train_losses):
    # Prepare DataFrame for seaborn
    epochs = list(range(1, len(train_losses) + 1))
    data = pd.DataFrame(
        {
            "Epoch": epochs,
            "Loss": train_losses,
        }
    )

    # Plot with seaborn
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=data, x="Epoch", y="Loss", marker="o")
    plt.title("Training Loss (Full Dataset)")
    plt.tight_layout()
    plt.show()

# Only Images


In [ ]:
train_hdf5_loader = DataLoader(
    train_hdf5_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,
    num_workers=NUM_WORKERS,
)

valid_hdf5_loader = DataLoader(
    valid_hdf5_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
)

test_hdf5_loader = DataLoader(
    test_hdf5_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
)

full_hdf5_dataset = ConcatDataset([train_hdf5_dataset, valid_hdf5_dataset])
full_hdf5_loader = DataLoader(
    full_hdf5_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
)

print(
    f"Train loader: {len(train_hdf5_loader)} batches (total = {NUM_SAMPLES} samples / {BATCH_SIZE} batches)"
)
print(f"Valid loader: {len(valid_hdf5_loader)} batches")
print(f"Test loader:  {len(test_hdf5_loader)} batches")
print(f"Full loader:  {len(full_hdf5_loader)} batches")

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim=32):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.fc_mu = nn.Linear(64 * 28 * 28, latent_dim)
        self.fc_logvar = nn.Linear(64 * 28 * 28, latent_dim)

        self.decoder_fc = nn.Linear(latent_dim, 64 * 28 * 28)
        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1),
            nn.Sigmoid(),  # to ensure output values are between 0 and 1
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        x = x.view(x.size(0), -1)
        h1 = self.encoder(x)
        mu = self.fc_mu(h1)
        logvar = self.fc_logvar(h1)
        z = self.reparameterize(mu, logvar)
        h2 = self.decoder_fc(z)
        x_reconstructed = self.decoder_conv(h2)
        return x_reconstructed, mu, logvar

    def loss_function(self, recon_x, x, mu, logvar):
        BCE = nn.functional.binary_cross_entropy(
            recon_x, x.view(-1, 3, 224, 224), reduction="sum"
        )
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + KLD

In [ ]:
model = VAE()

stopping_epochs = train_valid(model, train_hdf5_loader, valid_hdf5_loader, patience=5)

In [ ]:
final_model = VAE()

train_eval(
    final_model,
    full_hdf5_loader,
    test_hdf5_loader,
    early_stopping_epochs=stopping_epochs,
    output_model_file="/kaggle/working/VAE_hdf5.pth",
    output_submission_file="/kaggle/working/VAE_hdf5.csv",
)